In [132]:
import pandas as pd
import numpy as np

In Lab 1, we briefly touched on dataframes and some of the manipulation of which they are capable. In order to begin the data cleaning and scaling/standardization processes - part of data pre-processing - we will need the capabilities provided us by dataframces.

Again, we will depart slightly from the R-based Lab 2 in that we will read in the entire database and then use filtering to keep only the rows we want for demonstration purposes.

## Parsing the File

The file has no columns names (header), so as you saw in Lab 1, we can read in the data and establish the header simultaneously.

### Method 1: Step-by-step

We will first build the dataframe from the data we read in slowly. This is for those who, among other reasons, might want to practice with pandas dataframes a little more or just understand a little better what happens in the "condensed" path taken by Method 2 below.

In [6]:
fullData = pd.read_csv("103.txt", header=None)
fullData.head()

,0,1,2,3,4,5,6,7,8,9
0,Position_Introhouse,103,0.8,450.19,-132.86,0.61,0.00,-0.0,256.03,1.0
1,Position_Introhouse,103,1.0,450.19,-132.86,0.61,0.00,-0.0,256.03,1.0
2,Quest,103,21.78,AAAPaulDia,10,NaN,NaN,NaN,NaN,NaN
3,Quest,103,21.78,AAAPaulDia,Started,NaN,NaN,NaN,NaN,NaN
4,Dialogue,103,67.39,1,AAAPaul,Where am I?.,NaN,NaN,NaN,NaN


There's a lot of extra data here. More experience programmers could do the filtering in one fell swoop. To make it easier to follow along, we will break it down step-by-step.

We only want rows with data on Quests. So let's filter the fullData into only rows with quests. Those rows have the value of 'Quest' in the first column.

In [71]:
questData1 = fullData[(fullData[0]=="Quest")]
questData1.head()

,0,1,2,3,4,5,6,7,8,9
2,Quest,103,21.78,AAAPaulDia,10,NaN,NaN,NaN,NaN,NaN
3,Quest,103,21.78,AAAPaulDia,Started,NaN,NaN,NaN,NaN,NaN
179,Quest,103,156.99,AAAPaulDia,100,NaN,NaN,NaN,NaN,NaN
180,Quest,103,156.99,AAAPaulDia,Completed,NaN,NaN,NaN,NaN,NaN
205,Quest,103,197.21,AAAMrsWDia,10,NaN,NaN,NaN,NaN,NaN


There's still alot of unnecessary data here. There other action rows needed columns 5-9. Quests do not. Let's drop them.

In [27]:
questData2 = questData1.drop(questData1.loc[:, 5:], axis=1)
questData2.head()

,0,1,2,3,4
2,Quest,103,21.78,AAAPaulDia,10
3,Quest,103,21.78,AAAPaulDia,Started
179,Quest,103,156.99,AAAPaulDia,100
180,Quest,103,156.99,AAAPaulDia,Completed
205,Quest,103,197.21,AAAMrsWDia,10


Alternatively, instead of dropping the unnecesary columns, we could have copied the necessary columns. Since we also do not need the first two columns - 0 and 1 - let's slice the last three columns

In [107]:
questData3 = questData2.iloc[:, 2:]
questData3.head()

,2,3,4
2,21.78,AAAPaulDia,10
3,21.78,AAAPaulDia,Started
179,156.99,AAAPaulDia,100
180,156.99,AAAPaulDia,Completed
205,197.21,AAAMrsWDia,10


Now we have the three columns we want. Let's do a little further prep work. First, we'll name the columns to make it easier to work with them.

In [108]:
questData3 = questData3.rename(columns={2: "Quest_Timestamp", 3: "Quest_Name", 4: "Quest_Status"})
questData3.head()

,Quest_Timestamp,Quest_Name,Quest_Status
2,21.78,AAAPaulDia,10
3,21.78,AAAPaulDia,Started
179,156.99,AAAPaulDia,100
180,156.99,AAAPaulDia,Completed
205,197.21,AAAMrsWDia,10


One last thing - to obtain the same order as in the R labs, we'll move the first column and make it last. Just copy the dataframe to itself with the columns in the right order,

In [109]:
questData3 = questData3[["Quest_Name", "Quest_Status", "Quest_Timestamp"]]
questData3.head()

,Quest_Name,Quest_Status,Quest_Timestamp
2,AAAPaulDia,10,21.78
3,AAAPaulDia,Started,21.78
179,AAAPaulDia,100,156.99
180,AAAPaulDia,Completed,156.99
205,AAAMrsWDia,10,197.21


Lastly, always be wary of columns where there are strings or string-like objects. Often, in moving from file to file or just moving data around, some errant little spaces sneak in, so it's good to make sure you strip the out. Often, you cannot tell that they are there and you will spin in circles trying to catch errors later on.

In [110]:
questData3["Quest_Status"] = questData3["Quest_Status"].str.strip()

In [111]:
questData4 = questData3[questData3["Quest_Status"].isin(["Started", "Completed"])]
questData4

,Quest_Name,Quest_Status,Quest_Timestamp
3,AAAPaulDia,Started,21.78
180,AAAPaulDia,Completed,156.99
206,AAAMrsWDia,Started,197.21
1512,AAAMrWDia,Started,602.82
1539,AAAMrWDia,Completed,620.60
1614,AAAMrWDia,Completed,647.83
1616,AAAJohnsonDia,Started,661.03
1621,AAAJohnsonDia,Started,709.04
1626,MyNewQuest,Started,754.64
3565,aaamut,Started,1319.28


### Method 2: All at once

Now, let's look at another way to do it!

Note: At the time of writing this code, "append" was still usable but warnings would appear about imminent deprecation and recommending "concat" instead. Hence, the choice of "concat" below.

In [131]:
#Create a database with the column names in the order that we want them
questData5 = pd.DataFrame(columns=["Quest_Name", "Quest_Status", "Quest_Timestamp"])

#First, we open the file for reading
with open("103.txt", "r") as f:
    Lines = f.readlines()
    for line in Lines:
        #Next, split each line into a list of the elements
        words = line.split(",")
        #Check to see if the first element indicates the line is about a quest AND if so, check if the status column is "Started" or "Completed"
        if ((words[0].strip() == "Quest") and (words[4].strip() in (["Started", "Completed"]))):
            #If so, the create a new row, assigning each element to the proper column
            new_row = pd.DataFrame({"Quest_Name":words[3].strip(), "Quest_Status":words[4].strip(), "Quest_Timestamp":words[2].strip()}, index=[0])
            #Finally, conactenate that row to the datafame, one-by-one.
            questData5 = pd.concat([new_row,questData5.loc[:]]).reset_index(drop=True)
f.close()
questData5

,Quest_Name,Quest_Status,Quest_Timestamp
0,AAAPaulDia,Completed,2483.38
1,AAAPaulDia,Completed,2462.18
2,AAAMrsWDia,Completed,2330.61
3,AAAMrWDia,Started,2138.24
4,AAAPaulDia,Completed,2105.84
5,AAAPaulDia,Completed,2097.65
6,AAAcraft,Started,1978.27
7,aaamut,Completed,1532.04
8,aaamut,Started,1319.28
9,MyNewQuest,Started,754.64


There we have it! The second method is certainly shorter and straight to the point - but we still accomplish every step we did up above!